In [13]:
from IPython.display import display, Image, clear_output, HTML
import time
import random
random.seed(1) #makes the test reproducible
import keyboard
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests

In [14]:
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [15]:
def test_instructions():

    
    data_consent_info = """DATA CONSENT INFORMATION:

Please read:

We wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please type   yes   in the box below if you consent to the upload."""

    print(data_consent_info)
    result = input("> ") 

    if result == "yes": 
        print("Thanks for your participation.")
        print("Please contact a.fedorec@ucl.ac.uk")
        print("If you have any questions or concerns")
        print("regarding the stored results.")
    
    else: 
        # end code execution by raising an exception
        raise(Exception("User did not consent to continue test."))
        
    id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifier would be CBTC
"""

    print(id_instructions)
    user_id = input("> ")

    print("User entered id:", user_id)
    
    enter_gender = '''
    
Enter your gender.

Enter male, female or other.'''
    
    print(enter_gender)
    gender = input('> ')
    
    if gender.lower() in ("male", "female", "other"):
        print("User entered gender:", gender)
    else: 
        # end code execution by raising an exception
        raise(Exception("User did not enter a gender."))
    
    
    
    

    style = "color : green; font-size : 60px;"
    first_instruction = HTML(f'<h1 style = {style}>Welcome to the Approximate Number Sense test. A series of images with blue and yellow dots will flash on the screen.</h1>')
    second_instruction = HTML(f'<h1 style = {style}>Press the left arrow button ON THE KEYBOARD if the left side has more dots, otherwise press the right arrow button if the right side has more dots. You have 3 seconds to answer FROM THE MOMENT THE IMAGE FLASHES ON THE SCREEN. Answer as quickly and accurately as possible.</h1>')
    display(first_instruction)
    time.sleep(4)
    clear_output(wait = False)
    display(second_instruction)
    time.sleep(7)
    clear_output(wait = False)
    
    return user_id, gender

In [16]:
def correct_or_incorrect(image, images_dict):
    
    if keyboard.is_pressed('left'):
        if images_dict[image] == 'left':
            correct = True
        elif images_dict[image] == 'right':
            correct = False
                    
    elif keyboard.is_pressed('right'):
        if images_dict[image] == 'right':
            correct = True
        elif images_dict[image] == 'left':
            correct = False
                     
    else:
        correct = None
          
    return correct
    

In [17]:
def ans_test(images_list, images_dict, empty_image):
    
    total_correct = 0
    total_incorrect = 0
    total_time = 0
    
    results_dict = {'filename': [], 'correct': [], 'time': [], 'total_time': [], 'total_correct': [], 'total_incorrect': []}
    filename_list = []
    
    for image in images_list:
        
        image_name = image.filename
        results_dict['filename'].append(image_name)
        
        time_taken = 0
        start_time = time.time()
        
        correct = None
        display(image)
        
        while time.time() - start_time <= 0.75 and correct == None:
            correct = correct_or_incorrect(image, images_dict)
            
            
        clear_output(wait = False)
        
        if correct != None:
            time_taken = time.time() - start_time
            results_dict['time'].append(time_taken)
            
        else:
            display(empty_image)
            
            while (time.time() - start_time <= 2.75) and correct == None:
                correct = correct_or_incorrect(image, images_dict)
                
                
            clear_output(wait = False)
            
            if time_taken == 0:
                time_taken = time.time() - start_time
                results_dict['time'].append(time_taken)
                
        total_time += time_taken
            
        if correct == True:
            total_correct += 1
            results_dict['correct'].append('correct')
        else:
            total_incorrect += 1
            results_dict['correct'].append('incorrect')
            
        
        time.sleep(1.5)
    
    results_dict['total_time'] = total_time
    results_dict['total_correct'] = total_correct
    results_dict['total_incorrect'] = total_incorrect
    print(f'Your score was {total_correct} out of 64.')
    
    return results_dict

In [18]:
def main():
    empty_image = Image(filename = 'empty_image.png', width = 1000) #currently 40 pics, probably needs 64 (so 24 more)
    
    image_9_12_r1 = Image(filename = '9_12_r1.png', width = 1000)
    image_9_12_r2 = Image(filename = '9_12_r2.png', width = 1000)
    image_9_12_r3 = Image(filename = '9_12_r3.png', width = 1000)
    image_9_12_r4 = Image(filename = '9_12_r4.png', width = 1000)
    
    image_12_9_l1 = Image(filename = '12_9_l1.png', width = 1000)
    image_12_9_l2 = Image(filename = '12_9_l2.png', width = 1000)
    image_12_9_l3 = Image(filename = '12_9_l3.png', width = 1000)
    image_12_9_l4 = Image(filename = '12_9_l4.png', width = 1000)
    
    image_14_12_l1 = Image(filename = '14_12_l1.png', width = 1000)
    image_14_12_l2 = Image(filename = '14_12_l2.png', width = 1000)
    image_14_12_l3 = Image(filename = '14_12_l3.png', width = 1000)
    image_14_12_l4 = Image(filename = '14_12_l4.png', width = 1000)
    
    image_12_14_r1 = Image(filename = '12_14_r1.png', width = 1000)
    image_12_14_r2 = Image(filename = '12_14_r2.png', width = 1000)
    image_12_14_r3 = Image(filename = '12_14_r3.png', width = 1000)
    image_12_14_r4 = Image(filename = '12_14_r4.png', width = 1000)
    
    image_15_20_r1 = Image(filename = '15_20_r1.png', width = 1000)
    image_15_20_r2 = Image(filename = '15_20_r2.png', width = 1000)
    image_15_20_r3 = Image(filename = '15_20_r3.png', width = 1000)
    image_15_20_r4 = Image(filename = '15_20_r4.png', width = 1000)
    
    image_20_15_l1 = Image(filename = '20_15_l1.png', width = 1000)
    image_20_15_l2 = Image(filename = '20_15_l2.png', width = 1000)
    image_20_15_l3 = Image(filename = '20_15_l3.png', width = 1000)
    image_20_15_l4 = Image(filename = '20_15_l4.png', width = 1000)
    
    image_18_16_l1 = Image(filename = '18_16_l1.png', width = 1000)
    image_18_16_l2 = Image(filename = '18_16_l2.png', width = 1000)
    image_18_16_l3 = Image(filename = '18_16_l3.png', width = 1000)
    image_18_16_l4 = Image(filename = '18_16_l4.png', width = 1000)
    
    image_16_18_r1 = Image(filename = '16_18_r1.png', width = 1000)
    image_16_18_r2 = Image(filename = '16_18_r2.png', width = 1000)
    image_16_18_r3 = Image(filename = '16_18_r3.png', width = 1000)
    image_16_18_r4 = Image(filename = '16_18_r4.png', width = 1000)
    
    image_20_18_l1 = Image(filename = '20_18_l1.png', width = 1000)
    image_20_18_l2 = Image(filename = '20_18_l2.png', width = 1000)
    image_20_18_l3 = Image(filename = '20_18_l3.png', width = 1000)
    image_20_18_l4 = Image(filename = '20_18_l4.png', width = 1000)
    
    image_18_20_r1 = Image(filename = '18_20_r1.png', width = 1000)
    image_18_20_r2 = Image(filename = '18_20_r2.png', width = 1000)
    image_18_20_r3 = Image(filename = '18_20_r3.png', width = 1000)
    image_18_20_r4 = Image(filename = '18_20_r4.png', width = 1000)
    
    image_21_18_l1 = Image(filename = '21_18_l1.png', width = 1000)
    image_21_18_l2 = Image(filename = '21_18_l2.png', width = 1000)
    image_21_18_l3 = Image(filename = '21_18_l3.png', width = 1000)
    image_21_18_l4 = Image(filename = '21_18_l4.png', width = 1000)
    
    image_18_21_r1 = Image(filename = '18_21_r1.png', width = 1000)
    image_18_21_r2 = Image(filename = '18_21_r2.png', width = 1000)
    image_18_21_r3 = Image(filename = '18_21_r3.png', width = 1000)
    image_18_21_r4 = Image(filename = '18_21_r4.png', width = 1000)
    
    image_9_10_r1 = Image(filename = '9_10_r1.png', width = 1000)
    image_9_10_r2 = Image(filename = '9_10_r2.png', width = 1000)
    image_9_10_r3 = Image(filename = '9_10_r3.png', width = 1000)
    image_9_10_r4 = Image(filename = '9_10_r4.png', width = 1000)
    
    image_10_9_l1 = Image(filename = '10_9_l1.png', width = 1000)
    image_10_9_l2 = Image(filename = '10_9_l2.png', width = 1000)
    image_10_9_l3 = Image(filename = '10_9_l3.png', width = 1000)
    image_10_9_l4 = Image(filename = '10_9_l4.png', width = 1000)
    
    image_16_12_l1 = Image(filename = '16_12_l1.png', width = 1000)
    image_16_12_l2 = Image(filename = '16_12_l2.png', width = 1000)
    image_16_12_l3 = Image(filename = '16_12_l3.png', width = 1000)
    image_16_12_l4 = Image(filename = '16_12_l4.png', width = 1000)
    
    image_12_16_r1 = Image(filename = '12_16_r1.png', width = 1000)
    image_12_16_r2 = Image(filename = '12_16_r2.png', width = 1000)
    image_12_16_r3 = Image(filename = '12_16_r3.png', width = 1000)
    image_12_16_r4 = Image(filename = '12_16_r4.png', width = 1000)
    
    all_images = [image_9_12_r1, image_9_12_r2, image_9_12_r3, image_9_12_r4, image_12_9_l1, image_12_9_l2, image_12_9_l3, image_12_9_l4,
                  image_14_12_l1, image_14_12_l2, image_14_12_l3, image_14_12_l4, image_12_14_r1, image_12_14_r2, image_12_14_r3, image_12_14_r4,
                  image_15_20_r1, image_15_20_r2, image_15_20_r3, image_15_20_r4, image_20_15_l1, image_20_15_l2, image_20_15_l3, image_20_15_l4,
                  image_18_16_l1, image_18_16_l2, image_18_16_l3, image_18_16_l4, image_16_18_r1, image_16_18_r2, image_16_18_r3, image_16_18_r4,
                  image_20_18_l1, image_20_18_l2, image_20_18_l3, image_20_18_l4, image_18_20_r1, image_18_20_r2, image_18_20_r3, image_18_20_r4,
                  image_21_18_l1, image_21_18_l2, image_21_18_l3, image_21_18_l4, image_18_21_r1, image_18_21_r2, image_18_21_r3, image_18_21_r4,
                  image_16_12_l1, image_16_12_l2, image_16_12_l3, image_16_12_l4, image_12_16_r1, image_12_16_r2, image_12_16_r3, image_12_16_r4,
                  image_9_10_r1, image_9_10_r2, image_9_10_r3, image_9_10_r4, image_10_9_l1, image_10_9_l2, image_10_9_l3, image_10_9_l4]
    
    images_list = all_images.copy()
    random.shuffle(images_list)

    images_dict = {image_9_12_r1 : 'right', image_9_12_r2 : 'right', image_9_12_r3 : 'right', image_9_12_r4 : 'right', 
                   image_12_9_l1 : 'left', image_12_9_l2 : 'left', image_12_9_l3 : 'left', image_12_9_l4 : 'left',
                   image_14_12_l1 : 'left', image_14_12_l2 : 'left', image_14_12_l3 : 'left',  image_14_12_l4 : 'left',
                   image_12_14_r1 : 'right', image_12_14_r2 : 'right', image_12_14_r3 : 'right', image_12_14_r4 : 'right',
                   image_15_20_r1 : 'right', image_15_20_r2 : 'right', image_15_20_r3 : 'right', image_15_20_r4 : 'right',
                   image_20_15_l1 : 'left', image_20_15_l2 : 'left', image_20_15_l3 : 'left', image_20_15_l4 : 'left',
                   image_18_16_l1 : 'left', image_18_16_l2 : 'left', image_18_16_l3 : 'left',  image_18_16_l4 : 'left',
                   image_16_18_r1 : 'right', image_16_18_r2 : 'right', image_16_18_r3 : 'right', image_16_18_r4 : 'right',
                   image_20_18_l1 : 'left', image_20_18_l2 : 'left', image_20_18_l3 : 'left', image_20_18_l4 : 'left',
                   image_18_20_r1 : 'right', image_18_20_r2 : 'right', image_18_20_r3 : 'right', image_18_20_r4 : 'right',
                   image_18_21_r1 : 'right', image_18_21_r2 : 'right', image_18_21_r3 : 'right', image_18_21_r4 : 'right', 
                   image_21_18_l1 : 'left', image_21_18_l2 : 'left', image_21_18_l3 : 'left',  image_21_18_l4 : 'left',
                   image_9_10_r1 : 'right', image_9_10_r2 : 'right', image_9_10_r3 : 'right', image_9_10_r4 : 'right', 
                   image_10_9_l1 : 'left', image_10_9_l2 : 'left', image_10_9_l3 : 'left', image_10_9_l4 : 'left',
                   image_16_12_l1 : 'left', image_16_12_l2 : 'left', image_16_12_l3 : 'left', image_16_12_l4 : 'left',
                   image_12_16_r1 : 'right', image_12_16_r2 : 'right', image_12_16_r3 : 'right', image_12_16_r4 : 'right'}

    user_id, gender = test_instructions()
    results_dict = ans_test(images_list, images_dict, empty_image)
    
    combined_dict = {'user_id': user_id, 'gender': gender}
    combined_dict.update(results_dict)
    
    mydataframe = pd.DataFrame(combined_dict)
    myjson = mydataframe.to_json
    
    #print(combined_dict)
    
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSdNAOKUy2gKIWuvByvbJV8gLC7YTaaAxeKAq_7BEiw0TErzRw/viewform?usp=sf_link'
    #send_to_google_form(myjson, form_url)
    send_to_google_form(combined_dict, form_url)
    
    print('The End. Thank you :>')

                    
if __name__ == '__main__':
    main()

Your score was 54 out of 64.
The End. Thank you :>
